In [1]:
## add ignore warnings for now, will remove and debug once full algorithm is complete
import warnings
warnings.filterwarnings("ignore")

## import packages/libraries
from time import perf_counter, clock_gettime_ns, CLOCK_REALTIME
import numpy as np
from multiprocessing import Pool, cpu_count
from itertools import product
import sys
import sqlite3

## append filepath to allow files to be called from within project folder
sys.path.append('/home/gerard/Desktop/capstone_project/patoms')
sys.path.append('/home/gerard/Desktop/capstone_project')

## call locally created functions
from snapshot_2d_pattern_v5 import patoms2d
from snapshot_3d_pattern_v6 import patoms3d
from pattern_2d_compare_v3 import pattern_compare_2d
from pattern_3d_compare_v4 import pattern_compare_3d
from get_2d_table_names_v0 import table_names_2d
from updating_ref_table_v0 import update_ref_table
from patom_to_table_v0 import patom_to_table_func

In [2]:
## create in memory 2d database
con2d = sqlite3.connect(":memory:")
cur2d = con2d.cursor()
cur2d.execute('create table pat_2d_000000(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);')

## set similarity threshold limits
xc_perc = 0.80 
yc_perc = 0.80
xp_perc = 0.80
xn_perc = 0.80
yp_perc = 0.80
yn_perc = 0.80

In [ ]:
## read 2d ref database into memory and convert to ref patoms for comparison
working_ref_patoms = []
with open('/home/gerard/Desktop/capstone_project/ref2d.db', 'rb') as f:
    con2dref = sqlite3.connect(":memory:")
    con2dref.deserialize(f.read())
    cur2dref = con2dref.cursor()
    ref_names = [name for (name,) in cur2dref.execute("select name from sqlite_master where type='table';").fetchall()]
    for i in ref_names:
        table = cur2dref.execute(f"select * from {i};").fetchall()
        # patom = [norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, patom_ind, frame_ind, patom_time]
        norm_x = [x[0] for x in table]; norm_y = [x[1] for x in table]; pattern_centroid_x = [x[2] for x in table][0]; pattern_centroid_y = [x[3] for x in table][0]
        working_ref_patoms.append([norm_x, norm_y, pattern_centroid_x, pattern_centroid_y])

In [ ]:
val = 0
while val <= 30:
    rand_array = np.random.random((1, 720, 1280))
    y_len = rand_array.shape[1]
    x_len = rand_array.shape[2]
    # ingest data frame by frame
    for frame in range(rand_array.shape[0]):
        #################################################################################
        ####################### FIRST TASK: FIND PATTERNS IN FRAME ######################
        #################################################################################
        frame_patoms = patoms2d(x_len, y_len, rand_array[frame,:,:], frame)
        # patom = [[norm_x, norm_y], [pattern_centroid_x, pattern_centroid_y], patom_ind, frame_ind, patom_time]
        # patom[i][[0][0]: list of x_pos, patom[i][[0][1]: list of y_pos, patom[i][[1][0]: x_cent, patom[i][[1][1]: y_cent, patom[i][[2]: patom_ind, patom[i][[3]: frame_ind, patom[i][[4]: patom_time,
        num_patoms = len(frame_patoms)
        val += 1
        ###########################################################################
        ########## SECOND TASK: COMPARE NEW PATOMS AGAINST REF PATOMS #############
        ###########################################################################
        comp_indexes = list(product(range(num_patoms), range(len(working_ref_patoms))))
        with Pool(processes=cpu_count()) as pool:
            items = [(frame_patoms[i[0]][0], frame_patoms[i[0]][1], frame_patoms[i[0]][2], frame_patoms[i[0]][3],\
                    working_ref_patoms[i[1]][0], working_ref_patoms[i[1]][1], working_ref_patoms[i[1]][2], working_ref_patoms[i[1]][3], i) for i in comp_indexes]
            ## function outputs ind value of the patom_indexes list, the centroid and distance similarity measures
            comp_results = pool.starmap(pattern_compare_2d, items)
            # res output: [xc_sim, yc_sim, sim_perc_xp, sim_perc_xn, sim_perc_yp, sim_perc_yn, (i, j)]
            ## loop through the output of the comparison function
            for ix, i in enumerate(comp_results):
                # get max table number from 2d database
                next_table_num = int(max([table for (table,) in cur2d.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()])[-6:]) + 1 
                next_table_num = str(next_table_num).zfill(6)
                # check if table exists, if so, add patom, if not create table and add patom
                ref_table_num = ref_names[i[2][1]][-6:] # check this is correct
                new_patom = frame_patoms[i[2][0]] # check this is correct
                patom_to_table = patom_to_table_func(new_patom)
                if (i[0][0] >= xc_perc) and (i[0][1] >= yc_perc) and (i[1][0] >= xp_perc) and (i[1][1] >= xn_perc) and (i[1][2] >= yp_perc) and (i[0][3] >= yn_perc):
                    # add patom to relevant pattern table get name of ref table to get data table name
                    print('similar')
                    if f"pat_2d_{ref_table_num}" in [names for (names,) in cur2d.execute("select name from sqlite_master where type='table';").fetchall()]:
                        cur2d.executemany(f"INSERT INTO pat_2d_{ref_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
                        con2d.commit()
                        # patom = [norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, patom_ind, frame_ind, patom_time]
                    else:
                        cur2d.execute(f"CREATE TABLE pat_2d_{ref_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);")
                        cur2d.executemany(f"INSERT INTO pat_2d_{ref_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
                        con2d.commit()
                else:
                    # if patom is not matched then create new pattern & ref tables, get max table name and then create next new one
                    cur2d.execute(f"CREATE TABLE pat_2d_{next_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);")
                    cur2d.executemany(f"INSERT INTO pat_2d_{next_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
                    con2d.commit()


In [ ]:
# patom_table_names = cur2d.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE '%ref%';").fetchall()
# patom_tables = [name for (name,) in patom_table_names]
# for (name,) in patom_table_names:
#     print(cur2d.execute(f"select count(distinct patom_time) from {name};").fetchone())

# con2d.close()